In [ ]:
pip install openpyxl

In [ ]:
pip install xlsxwriter

In [ ]:
pip install lda

In [ ]:
import sys
sys.path.append('../input')
from xuexibao.utils.fileUtils import *
import lda
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from stop_words import get_stop_words
import spacy 
nlp = spacy.load('en_core_web_sm')
stop_words = list(get_stop_words('english'))
import dill
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel
from matplotlib.pyplot import MultipleLocator
import matplotlib.pyplot as plt
import pickle

In [ ]:
lines = readExcelToList('../input/fcdc55/f4.2.xlsx')

In [ ]:
corpus = [i[3] for i in lines][1:]
stop_list2 = set(['facebook','I','app','people','can','will','like','?','want','make','see','get','even','know','just','go','FB','also','re','many','ve','say',
             'one','much','bad','think','year','fb','away','post','now','facebook','need','look','now','well','set','human','without','party','ever',
             'give','way','everything','ask','medium','now','company','thing','try','standard','become','every','new','user','big','put','take','party','good',
              'take','phone','friend','something','fact','someone','family','stop','since','always','nothing','anything','long','still','talk','tell','old','call','etc',
              'everyone','show','never','let','anyone','able','day','come','really','Facebook','anymore','39','real','amp'
             ])
non_stop = set(['know','say','bad','think','post','need','like','look','set','well','human','party','good','phone','friend','fact','family','stop','talk', 'tell','old','call'])
stop = set(['day', 'month', 'year','time'])
stop_list2 = list(stop_list2|stop-non_stop)
for i in range(len(corpus)):
    tmp = ' '.join([token.lemma_ for token in nlp(corpus[i]) if len(token.lemma_)>2 and token.is_digit==False and token.is_punct==False and token.lemma_ not in stop_words and
                   token.lemma_ not in stop_list2])
    corpus[i] = tmp.replace('info ','information ').replace('39','').replace('facebook','')

In [ ]:
texts = []
for i in corpus:
    texts.append(i.split(' '))

In [ ]:
from collections import Counter
ta = []
for i in texts:
    ta.extend(i)
wd = Counter(ta)
print(wd.most_common(300))

In [ ]:
def test1(num):
    a = list(range(3,num+1))
    b = []
    texts = []
    for i in corpus:
        texts.append(i.split(' '))
    common_dictionary = corpora.Dictionary(texts)
    common_corpus = [common_dictionary.doc2bow(text) for text in texts]
    for i in range(3,num+1):
        model = LdaModel(common_corpus, i, common_dictionary,random_state=1,iterations=1500,alpha=0.1,eta=0.01,passes=20)
        cm=CoherenceModel(model=model, texts=texts,dictionary=common_dictionary,  coherence='c_v')
        coherence = cm.get_coherence()  # get coherence value
        b.append(coherence)
    x_major_locator = MultipleLocator(1)
    ax = plt.gca()
    ax.xaxis.set_major_locator(x_major_locator)  
    plt.plot(a,b)
    plt.show()
    plt.savefig('c_v.png')
    return a,b
test1(20)

In [ ]:
def model_display(num):
    texts = []
    for i in corpus:
        texts.append(i.split(' '))
    common_dictionary = corpora.Dictionary(texts)
    common_corpus = [common_dictionary.doc2bow(text) for text in texts]
    #model = LdaModel(common_corpus, 5, common_dictionary,random_state=1,iterations=1500,alpha=0.1,eta=0.01)
    model = LdaModel(common_corpus, num, common_dictionary,random_state=1,iterations=1500,alpha=0.1,eta=0.01,passes=20)
    DocumentTopicMatrix = np.zeros([len(texts), num])
    for id in range(len(common_corpus)):
        doc_top = model.get_document_topics(common_corpus[id], per_word_topics=False)
        m = 0
        for x in doc_top:
            DocumentTopicMatrix[id][m] = round(x[1],4)
            m += 1
    return DocumentTopicMatrix,model

In [ ]:
DocumentTopicMatrix,model = model_display(8)

In [ ]:
len(DocumentTopicMatrix)

In [ ]:
pickle.dump(DocumentTopicMatrix,open('DocumentTopicMatrix.pk','wb'))
pickle.dump(model,open('model.pk','wb'))

In [ ]:
def save(num):
    texts = []
    for i in corpus:
        texts.append(i.split(' '))
    common_dictionary = corpora.Dictionary(texts)
    common_corpus = [common_dictionary.doc2bow(text) for text in texts]
    #model = LdaModel(common_corpus, 5, common_dictionary,random_state=1,iterations=1500,alpha=0.1,eta=0.01)
    model = LdaModel(common_corpus, num, common_dictionary,random_state=1,iterations=1500,alpha=0.1,eta=0.01,passes=20)
    t = model.show_topics(num_words=21)
    t = [i[1] for i in t]
    arr = []
    for id,s in enumerate(t):
        s =s.split('+')
        s = [i.split('*')[-1] for i in s]
        a = [i.strip().strip('"') for i in s]
        a.insert(0,'TOPIC'+str(id+1))
        arr.append(a)
    grid = arr
    grid = [[row[i] for row in grid] for i in range(len(grid[0]))]
    f = open('{}.csv'.format(num),'w')
    for i in grid:
        f.write(','.join(i)+'\n')
    f.close()

In [ ]:
save(8)